In [11]:
import cv2
import numpy as np
from keras.preprocessing.image import ImageDataGenerator
import tensorflow as tf
import pywt

In [12]:
def custom_preprocessing(img):
    img = (img * 255).astype(np.uint8)

    gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)

    coeffs = pywt.dwt2(gray, 'db1')  # Using Daubechies wavelet
    cA, (cH, cV, cD) = coeffs

    combined_edges = np.sqrt(cH**2 + cV**2 + cD**2)

    if combined_edges.min() != combined_edges.max():
        combined_edges_rescaled = ((combined_edges - combined_edges.min()) * (255 / (combined_edges.max() - combined_edges.min()))).astype(np.uint8)
    else:
        combined_edges_rescaled = np.zeros_like(combined_edges, dtype=np.uint8)

    wavelet_edges = cv2.resize(combined_edges_rescaled, (img.shape[1], img.shape[0]))

    stacked_img = np.stack([wavelet_edges] * 3, axis=-1)

    final_img = stacked_img.astype(np.float32) / 255.0

    final_img = np.clip(final_img, 0, 1)

    return final_img

train_datagen = ImageDataGenerator(
    preprocessing_function=custom_preprocessing,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

test_datagen = ImageDataGenerator(
    preprocessing_function=custom_preprocessing
)

train_dir = '/Users/tianze/cs4243_lab/miniproject/drive-download-20230925T130828Z-001/train/'
test_dir = '/Users/tianze/cs4243_lab/miniproject/drive-download-20230925T130828Z-001/test/'


train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(128, 128),
    batch_size=32,
    class_mode='binary'
)

test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=(128, 128),
    batch_size=32,
    class_mode='binary'
)
from keras.regularizers import l2

model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=(128, 128, 3)), #, kernel_regularizer=l2(0.0005)),  # Reduced regularization strength
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.MaxPooling2D((2, 2)),
    tf.keras.layers.Dropout(0.2),

    tf.keras.layers.Conv2D(64, (3, 3), activation='relu'), # kernel_regularizer=l2(0.0005)),  # Reduced regularization strength
    # tf.keras.layers.BatchNormalization(),
    tf.keras.layers.MaxPooling2D((2, 2)),
    tf.keras.layers.Dropout(0.3),

    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(256, activation='relu', kernel_regularizer=l2(0.0005)),  # Reduced neurons & regularization strength
    # tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(1, activation='sigmoid')
])
from keras.callbacks import EarlyStopping

model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

early_stopping = EarlyStopping(monitor='val_loss', patience=10, verbose=1, restore_best_weights=True)

history = model.fit(
    train_generator,
    steps_per_epoch=len(train_generator),
    epochs=7,
    validation_data= test_generator,
    validation_steps=len(test_generator),
    callbacks=[early_stopping]
)

model.save('weapon_detection_model.h5')

test_loss, test_acc = model.evaluate(test_generator)
print(f'Test accuracy: {test_acc}')

Found 2305 images belonging to 2 classes.
Found 1699 images belonging to 2 classes.
Epoch 1/7
73/73 [==============================] - 135s 2s/step - loss: 2.1095 - accuracy: 0.7423 - val_loss: 0.9885 - val_accuracy: 0.6886
Epoch 2/7
73/73 [==============================] - 132s 2s/step - loss: 0.7300 - accuracy: 0.8182 - val_loss: 0.8929 - val_accuracy: 0.6886
Epoch 3/7
73/73 [==============================] - 135s 2s/step - loss: 0.6597 - accuracy: 0.8148 - val_loss: 0.8238 - val_accuracy: 0.6886
Epoch 4/7
73/73 [==============================] - 138s 2s/step - loss: 0.6183 - accuracy: 0.8187 - val_loss: 0.7803 - val_accuracy: 0.6886
Epoch 5/7
73/73 [==============================] - 136s 2s/step - loss: 0.5803 - accuracy: 0.8174 - val_loss: 0.7661 - val_accuracy: 0.6886
Epoch 6/7
73/73 [==============================] - 148s 2s/step - loss: 0.5588 - accuracy: 0.8191 - val_loss: 0.9003 - val_accuracy: 0.6886
Epoch 7/7
73/73 [==============================] - 150s 2s/step - loss: 0.54

/Users/tianze/anaconda3/lib/python3.10/site-packages/keras/src/engine/training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


54/54 [==============================] - 56s 1s/step - loss: 0.9430 - accuracy: 0.6886
Test accuracy: 0.6886403560638428
